In [14]:
from pathlib import Path

import joblib
import pandas as pd
import xgboost as xgb

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import balanced_accuracy_score, multilabel_confusion_matrix
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.svm import LinearSVC
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
from sklearn import set_config
set_config(display='diagram')
%load_ext nb_black 

The nb_black extension is already loaded. To reload it, use:
  %reload_ext nb_black


<IPython.core.display.Javascript object>

## Read Data

In [23]:
df = pd.read_csv(
    Path(
        "sklearn",
        "assets",
        # "dataset.csv",
        "dataset-simple.csv",
    ),
    index_col='pvid',
)

df.head()

,category_level_1,category_level_2,storage_env,pack_type,cooking_type,label,text
pvid,,,,,,,
6345063,Grocery,Other Tins,Ambient,Can,['None'],other,Coco Fresh Coconut Milk 400ml. Coconut Milk. W...
6345067,Off Licence,Other Wines,Ambient,Bottle,['None'],none,Cockburn's Fine Tawny Port 75cl. Fine Tawny Po...
6345069,Drinks,Carbonated Drinks,Ambient,Plastic Bottle,['None'],drinks,Coca-Cola Zero Cherry 2 Litre. Sparkling Low C...
6345073,Off Licence,Red Wine,Ambient,Bottle,['None'],none,Co Op Fairtrade Carmenère 75cl. Carmenère - Re...
6345074,Drinks,Carbonated Drinks,Ambient,Can,['None'],drinks,Coca-Cola Zero Sugar Raspberry 250ml. Sparklin...


<IPython.core.display.Javascript object>

# Split data

- Use `label` as target labels
    - Number of classes is $63$
    - **Classes are very imbalanced**
        - Estimate class weights by using $n_{samples} / (n_{classes} * np.bincount(y))$ (not used but in case needed)
        - Calculate sample weights (not used but in case needed)
    - Stratify labels when splitting so their distribution in train/test data is similar
- Encode target labels with values between $0-62$
- Split $70/30$ for training/testing
- Create empty dict to store all classifiers. Once populated, an item will look like:
`'classifier name' : {
    'pipeline': ...,
    'params': ...,
    'best_score': ...,
    'best_params': ...,
    'best_estimator': ...,
    'best_estimator_params': ...,
    'testing_accuracy': ...,
    'testing_conf_matrix': ...,
}`

In [ ]:
y = df["label"]
num_class = len(y.unique())

class_weights = compute_class_weight(
    class_weight="balanced",
    classes=y.unique(),
    y=y,
)

sample_weights = compute_sample_weight(
    class_weight="balanced",
    y=y,
)

X = df.drop("label", axis=1)

le = LabelEncoder()
y = le.fit_transform(y)

if not Path("models").is_dir():
    Path("models").mkdir()

joblib.dump(
    le,
    Path(
        "sklearn",
        "models",
        "LabelEncoder.pkl",
    ),
)

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=42,
    shuffle=True,
    stratify=y,
)

classifiers = dict()

## Transformers

### Create 4 transformation `Pipeline`:

1. A pipeline for text features: `text`. This will have 1 step:
    - `TfidfVectorizer` which is equivalent to `CountVectorizer` followed by `TfidfTransformer`

2. A pipeline for categorical features with 1 label each: `category_level_1`, `category_level_2`, `storage_env`. This will have 1 step:
    - `OneHotEncoder`

3. A pipeline for categorical features with multiple labels each: `cooking_type`. This will have 1 step:
    - `CountVectorizer`

4. A pipeline for categorical features to be hashed: `pack_type`. This will have 1 step:
    - `FeatureHasher`. We used the hashing trick on `pack_type` as it has 45 categories and using `OneHotEncoder` would result in 45 sparse features
    - Use a power of 2 for `n_features`
    - Collisions are likely to cancel out rather than accumulate error when `alternate_sign=True`. However, MultinomialNB estimators expect non-negative inputs so will disable `alternate_sign`

- `remainder=drop` will be used to drop any extra features that might be added to the dataframe later as a safety guard. When adding new features, either pass them through a pipeline, or change to `remainder=passthrough`

In [ ]:
text_transformer = Pipeline(
    steps=[
        (
            "tfidf",
            TfidfVectorizer(
                lowercase=True, ngram_range=(1, 2), norm="l2", use_idf=True
            ),
        )
    ]
)

cat_transformer = Pipeline(
    steps=[
        (
            "onehot",
            OneHotEncoder(categories="auto", sparse=False, handle_unknown="ignore"),
        )
    ]
)

multi_cat_transformer = Pipeline(steps=[("binarizer", CountVectorizer(analyzer=set))])

hash_transformer = Pipeline(
    steps=[
        (
            "hasher",
            FeatureHasher(n_features=10, input_type="string", alternate_sign=False),
        )
    ]
)

preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_transformer, "text"),
        (
            "cat",
            cat_transformer,
            ["category_level_1", "category_level_2", "storage_env"],
        ),
        ("multi_cat", multi_cat_transformer, "cooking_type"),
        ("hash", hash_transformer, "pack_type"),
    ],
    remainder="drop",
)

### Check for Collisions

- Check for possible collisions in the hashing features for `pack_type`
- Increase `n_features` by 2 until no duplicated hashing features exist

In [ ]:
hashed_features = (
    hash_transformer.named_steps["hasher"].fit_transform(df["pack_type"]).toarray()
)

df_hashed_features = (
    df[["pack_type"]].reset_index(drop=True).join(pd.DataFrame(hashed_features))
)

df_hashed_features.groupby(["pack_type"]).first().duplicated(keep=False).sum()

# Classifiers

## Multinomial Naïve Bayes

- Normally requires bow
- tf-idf vectors are also known to work well in practice
- `alpha`: additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing)

In [ ]:
classifier = MultinomialNB(alpha=1.0)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier),
    ]
)

params = {
    'classifier__alpha': [0.001, 0.01, 0.1, 0.3, 0.5, 1],
}

classifiers['MultinomialNB'] = {
    'pipeline': pipeline,
    'params': params,
}

In [ ]:
pipeline

## Logistic Regression

- `multinomial` logistic regression yields more accurate results and is faster to train on larger scale dataset than `ovr` logistic regression
- `saga` solver works with `multinomial` and is faster for large datasets
- `penalty=l1` trims the weights of not informative features to zero which is good if the goal is to extract the strongly discriminative vocabulary of each class. However, to get the best predictive accuracy, it is better to use `penalty=l2` instead

In [ ]:
classifier = LogisticRegression(
    solver='saga',
    class_weight='balanced',
    multi_class='multinomial',
    penalty='l2',
    random_state=42,
    max_iter=100,
)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier),
    ]
)

params = {
    'classifier__C': [1, 10, 100],
}

classifiers['LogisticRegression'] = {
    'pipeline': pipeline,
    'params': params,
}


In [ ]:
pipeline

## Linear Support Vector Machine

- LinearSVC is Similar to SVC with parameter `kernel=linear` but scales better
to large number of samples
- Select `dual=True` when `n_samples` < `n_features`. The TfidfVectorizer creates a large sparse matrix as we are using uni-grams and bi-grams (more than 200K sparse features)
- Select `multi_class=ovr` to train one-vs-rest classifiers
- `class_weight=balanced`: automatically adjust weights inversely proportional to class frequencies
- `C`: regularization parameter. Higher values result in less regularization (narrower margin with fewer violations)
- Advantages
    - Good for linear and non-linear classification
    - Well suited for classification of complex but small to medium datasets
- Disadvantages
    - Speed and datasets size
    - Needs scaling to be centred around zero
    - Sensitive to outliers

In [ ]:
classifier = LinearSVC(
    penalty='l2',
    loss='squared_hinge',
    # dual=True,
    multi_class='ovr',
    class_weight='balanced',
    random_state=42,
    max_iter=1000,
)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier),
    ]
)

params = {
    'classifier__C': [1, 10, 100],
    'classifier__dual': [True, False],
}

classifiers['LinearSVC'] = {
    'pipeline': pipeline,
    'params': params,
}

In [ ]:
pipeline

## Random Forest Ensemble

- Each tree is trained on a random subset of the training set with replacement (i.e. bootstrap aggregating or bagging)
- Each tree is trained on a random subset of features, the number of features to use is defined by `max_features`
- Advantages
    - No need for scaling
    - No need for dimensionality reduction (unless rotation is needed)
- Disadvantages
    - Sensitive to small variations in the training data
    - Over-fitting
    - Difficult to interpret

In [ ]:
classifier = RandomForestClassifier(
    bootstrap=True,
    oob_score=True,
    max_features='sqrt',
    class_weight='balanced',
    random_state=42,
)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier),
    ]
)

params = {
    'classifier__criterion': ['gini', 'entropy'],
    'classifier__max_depth': [4, 5, 6],
    'classifier__n_estimators': [100, 200, 400],
}

classifiers['RandomForestClassifier'] = {
    'pipeline': pipeline,
    'params': params,
}

In [ ]:
pipeline

## Stochastic Extreme Gradient Boosting (XGBoost) Ensemble

- Boosting methods train predictors sequentially, each trying to correct its predecessor. Gradient Boosting tries to fit the new predictor to the residual error made by the previous one. Extreme Boosting is more regularized to control over-fitting, which gives it better performance
- XGBoost uses trees as the base booster by default (booster=`gbtree`), which has a sklearn API (other booster don't)
- `learning_rate`: scales the contribution of each tree (ASA shrinkage). Lower `learning_rate` requires more trees to fit the training data
- `subsample`: ratio of training data to be randomly sampled to train each tree (i.e. Stochastic). Typically, set `subsample >= 0.5` for good results when `sampling_method=uniform` which is the default
- `colsample_bytree`: ratio of columns to be randomly sampled prior to train each tree

In [ ]:
classifier = xgb.XGBClassifier(
    booster='gbtree',
    objective='multi:softmax',
    sampling_method='uniform',
    num_class=num_class,
    random_state=42,
)

pipeline = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('classifier', classifier),
    ]
)

params = {
    'classifier__learning_rate': [0.1, 0.3],
    'classifier__subsample': [0.7],
    'classifier__colsample_bytree': [0.7],
    'classifier__max_depth': [3, 4],
    'classifier__n_estimators': [100],
}

classifiers['XGBClassifier'] = {
    'pipeline': pipeline,
    'params': params,
}

In [ ]:
pipeline

# Training

## Grid search cross validation

- Use `balanced_accuracy` for scoring to deal with imbalanced classes
- Loop over each classifier in the classifiers dictionary
    - Train using training data
    - Pickle trained model

In [ ]:
for k, v in classifiers.items():

    print(f"\nRunning grid search with cross validation for {k}...")

    gs = GridSearchCV(
        v["pipeline"],
        v["params"],
        scoring="balanced_accuracy",
        cv=5,
        n_jobs=-1,
        verbose=2,
    )

    gs.fit(
        X_train,
        y_train,
    )

    joblib.dump(
        gs,
        Path(
            "sklearn",
            "models",
            f"{k}.pkl",
        ),
    )

# Validation

- Loop over each classifier in the classifiers dictionary
    - Load model
        - Store the grid search `best_score_`
        - Store the grid search `best_params_`
        - Store the grid search `best_estimator_`
        - Store the grid search `best_estimator_` params
    - Evalute using testing data
        - Store the `balanced_accuracy_score` (average recall obtained on each class to deal with imbalance classes)
        - Store class-wise `multilabel_confusion_matrix`

In [ ]:
for k, v in classifiers.items():

    gs = joblib.load(
        Path(
            "sklearn",
            "models",
            f"{k}.pkl",
        ),
    )

    v["best_score"] = gs.best_score_
    v["best_params"] = gs.best_params_
    v["best_estimator"] = gs.best_estimator_
    v["best_estimator_params"] = gs.best_estimator_.named_steps[
        "classifier"
    ].get_params()

    print(f"Running evaluation on test data for {k}...")
    y_pred = gs.predict(X_test)

    v["testing_accuracy"] = balanced_accuracy_score(
        y_test,
        y_pred,
    )

    v["testing_conf_matrix"] = multilabel_confusion_matrix(
        y_test,
        y_pred,
        samplewise=False,
    )

## Voting classifier

- Use `hard` voting ensamble of all the classifiers `best_estimator_`
    - Classifiers should be passed as a list of `(str, estimator)`
    - Train using training data-
        - Store training `best_score_`        
- Evalute using testing data
    - Store the `balanced_accuracy_score` (average recall obtained on each class to deal with imbalance classes)
    - Store class-wise `multilabel_confusion_matrix`
- Pickle the classifiers dictionary
- Balanced accuracy:
    - Training: $97\%$
    - Testing: $90\%$

In [ ]:
best_estimators = [(k, v["best_estimator"]) for k, v in classifiers.items()]

vc = VotingClassifier(
    estimators=best_estimators,
    voting="hard",
)

print("Fitting VotingClassifier using all best estimators...")
vc.fit(X_train, y_train)

y_pred = vc.predict(X_test)

classifiers["VotingClassifier"] = {
    "best_score": vc.score(X_train, y_train),
    "testing_accuracy": balanced_accuracy_score(
        y_test,
        y_pred,
    ),
    "testing_conf_matrix": multilabel_confusion_matrix(
        y_test,
        y_pred,
        samplewise=False,
    ),
}

joblib.dump(
    vc,
    Path(
        "sklearn",
        "models",
        "VotingClassifier.pkl",
    ),
)

### Save results as flat file

- Create DataFrame of results from all classifiers and save into cvs

In [ ]:
pd.DataFrame.from_dict(classifiers, orient="index",).to_csv(
    Path(
        "sklearn",
        "models",
        "models.csv",
    )
)